In [ ]:
# default_exp utils

In [ ]:
# hide
%load_ext autoreload
%autoreload 2

In [ ]:
# hide
from fastcore.test import *

# Utils

> Utilities for tracking the experimental results and saving the parameter setup

In [ ]:
# export

# For storing and Documentation
import datetime
import json
import datetime
import torch
import os
from tqdm import tqdm
from sklearn import metrics
from statistics import mean

# For Visualization
import matplotlib.pyplot as plt
import torchvision.utils as vutils
import numpy as np
from scipy.ndimage.interpolation import affine_transform

# Using tensorboard
from torch.utils.tensorboard import SummaryWriter

# name of the logfile
LOG = "/_Log.txt"

In [ ]:
# export


class Tracker(object):
    """
    The Data tracker class serves as a uniform datatracker for all Modules
    """

    def __init__(self, args, log_view=True):
        """
        Setup the Folders for tracking
        """
        super(Tracker, self).__init__()
        # general defines:
        self.channels = len(args.perspectives)
        self.dim = args.dim
        
        self.crop_size = args.crop_size
        self.pic_size = args.pic_size
        self.model_type = args.model_type
        self.log_view = log_view
        # Affine transform matrix.
        self.T = np.array([[1, -1], [0, 1]])
        # The results to track as empty dict:
        self.tr_dict = {}
        # create directory name
        date = datetime.datetime.now()
        self.dir_name = "../data/%s %d-%d-%d at %d-%d" % (
            args.model_type,
            date.year,
            date.month,
            date.day,
            date.hour,
            date.minute,
        )
        self.dataset = args.dataset_type

        # internal counting for visualization
        self.internal_count = 100
        self.classes = args.classes
        self.best_score = {}
        self.best_score["all"] = 0
        for cl in self.classes:
            self.best_score[cl] = 0
        # store the path of the best model
        self.model_path = self.dir_name + "/_model"
        self.image_path = self.dir_name + "/images"

        # repair for incorrect view
        self.view_re_x = [1, len(args.perspectives), -1, args.pic_size, args.pic_size]

        # Create new directory
        try:
            os.mkdir(self.dir_name)
            os.mkdir(self.image_path)
        except:
            print(
                "\nFoler: "
                + self.dir_name
                + " exists already.\nFiles will be overwritten.\n"
            )
            open(self.dir_name + LOG, "w").close()

        # Save set of hyperparameters
        with open(self.dir_name + "/_params.json", "w") as f:
            json.dump(args._get_kwargs(), f, ensure_ascii=False, indent=4)

    def view_3d(self, x, scale=0.7, alpha=1.0, bg_val=-1):
        """
        Visualize the 3d pictures as expanded slideshow
        args:
            x: 3d matrix (img_h, img_w, img_z)
            sclae: define the distance between slides
            alpha: visibility
            bg_val: background value
        """
        # List of images instead 3d matrix
        x = x.numpy()
        images = [x[i, :, :] for i in range(self.crop_size)]

        # Define size of new picture
        stacked_height = 2 * self.pic_size
        stacked_width = int(
            self.pic_size + (self.crop_size - 1) * self.pic_size * scale
        )
        stacked = np.full((stacked_height, stacked_width), bg_val)

        # Go over each slide
        for i in range(self.crop_size):
            # The first image will be right most and on the "bottom" of the stack.
            o = (self.crop_size - i - 1) * self.pic_size * scale
            out = affine_transform(
                images[i][0, :, :],
                self.T,
                offset=[o, -o],
                output_shape=stacked.shape,
                cval=bg_val,
            )
            stacked[out != bg_val] = out[out != bg_val]

        # plot the image series
        plt.imshow(stacked, alpha=alpha, interpolation="nearest", cmap="gray")

    def show(self, img):
        """subfunction for visualization"""
        npimg = img.numpy()
        plt.imshow(np.transpose(npimg, (1, 2, 0)), interpolation="nearest")
        
    def draw_loss(self, tr_data):
        """track the losses as a pdf plot"""
        # Go over all Losses and Plot
        plt.figure(figsize=(12, 12))
        for k in tr_data.keys():
            # Append / Create new
            if k in self.tr_dict.keys():
                self.tr_dict[k].append(tr_data[k])
            else:
                self.tr_dict[k] = [tr_data[k]]

            # Plot the current progress in "_Loss.png"
            plt.plot(self.tr_dict[k], label=k)

        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.yscale("log") if self.log_view else None
        plt.grid(True, alpha=0.25)
        plt.legend()
        plt.savefig(self.dir_name + "/_Losses.pdf")
        plt.close()
        
    def draw_mrnet(self, x, x_re, iteration):
        """draw an imagegrid for each channel"""
        
        for channel in range(self.channels):

            # init view list
            if self.dim == 3:

                if len(x.shape) < 5:
                    x = x.permute(1, 0, 2, 3)
                    x = x.reshape(self.view_re_x)

                if len(x_re.shape) < 5:
                    x_re = x_re.permute(1, 0, 2, 3)
                    x_re = x_re.reshape(self.view_re_x)

                real_pic = x[0, channel, :, :, :].view(
                    (self.crop_size, -1, self.pic_size, self.pic_size)
                )

                fake_pic = x_re[0, channel, :, :, :].view(
                    (self.crop_size, -1, self.pic_size, self.pic_size)
                )

                # add the 3d visualization
                plt.figure(figsize=(24, 12))
                plt.subplot(2, 1, 1)
                self.view_3d(real_pic)
                plt.subplot(2, 1, 2)
                self.view_3d(fake_pic)
                plt.savefig(
                    self.image_path
                    + "/rec_iteration_%d_class_%d_3d_view.png"
                    % (iteration, channel)
                )
                plt.close()
                
            else:
                real_pic = x[0, channel, :, :].view(
                    (self.crop_size, -1, self.pic_size, self.pic_size)
                )
                fake_pic = x_re[0, channel, :, :].view(
                    (self.crop_size, -1, self.pic_size, self.pic_size)
                )

            # plot in figure
            plt.figure(channel, figsize=(24, 24))
            if iteration == 0:
                self.show(
                    vutils.make_grid(real_pic, nrow=4, padding=2, normalize=True)
                )
                plt.savefig(self.image_path + "/class_%d__original.jpg" % (channel))
            else:
                self.show(
                    vutils.make_grid(fake_pic, nrow=4, padding=2, normalize=True)
                )
                plt.savefig(
                    self.image_path
                    + "/rec_iteration_%d_class_%d.jpg" % (iteration, channel)
                )
                plt.savefig(self.dir_name + "/current.jpg")
            plt.close()
            
    def draw_kneexray(self, x, x_re, iteration):
        """simple plot drawing the current picture"""
        # plot in figure
        plt.figure(1337, figsize=(24, 24))
        if iteration == 0:
            self.show(
                vutils.make_grid(x[0, :, :, :], nrow=4, padding=2, normalize=True)
            )
            plt.savefig(self.image_path + "/original.jpg")
        else:
            self.show(
                vutils.make_grid(
                    x_re[0, :, :, :], nrow=4, padding=2, normalize=True
                )
            )
            plt.savefig(self.image_path + f"/rec_iteration_{iteration}.jpg")
            plt.savefig(self.dir_name + "/current.jpg")
        plt.close()
    
    def track_progress(self, model, data, iteration):
        """
        Visualize the Progress by applying the validation loader and visualizing the results
        """
        # Get the original image
        x = model.prep(data).detach().cpu()

        # Get the current results:
        x_re, tr_data = model(data, update=False)
        x_re = x_re.detach().cpu()

        self.draw_loss(tr_data)

        # write to "_Log.txt" file
        with open(self.dir_name + LOG, "a") as file:
            file.write("\n\nIteration: %d" % (iteration))
            for k in tr_data.keys():
                file.write("\n%s: %.4f" % (k, tr_data[k]))

        # Save the losses as npy
        np.save(self.dir_name + "/_Losses.npy", self.tr_dict)

        # save the current model:
        torch.save(model.state_dict(), self.dir_name + "/_model")

        # Plot Results:
        if self.dataset == "MRNet":
            self.draw_mrnet(x, x_re, iteration)
            
        else:
            self.draw_kneexray(x, x_re, iteration)
            

    def update_network(self, cl, model, score):
        """
        update the network if the score is lower
        """
        if self.best_score[cl] < score:
            print("\n\n  ---Best Score %s %.4f---- \n" % (cl, score))
            # delete old model if it exists
            if self.best_score[cl] > 0:
                os.remove(self.model_path + "_%s_%.4f" % (cl, self.best_score[cl]))
            # save new model
            torch.save(
                model.state_dict(), self.model_path + "_%s_%.4f" % (cl, score)
            )
            # reset score
            self.best_score[cl] = score

    def get_accuracy(self, model, valid_loader, iteration):
        """
        Go trough the whole validation dataset and determine accuracy
        """
        # predefine
        preds = {}
        labels = {}
        for cl in self.classes:
            preds[cl] = []
            labels[cl] = []

        # calculate accuracy
        for data in valid_loader:
            pred, label, _ = model.forward(data, update=False)
            # get the current output
            pred = pred.data.cpu().numpy()
            label = label.data.cpu().numpy()
            # append predictions
            for i, cl in enumerate(self.classes):
                preds[cl].append(pred[i])
                labels[cl].append(label[i])

        # get the accuracies
        acc = {}
        for cl in self.classes:
            fpr, tpr, _ = metrics.roc_curve(labels[cl], preds[cl])
            loc_acc = metrics.auc(fpr, tpr)
            pred_label = [1 if pre > 0.5 else 0 for pre in preds[cl]]
            acc[cl] = accuracy_score(labels[cl], pred_label)
            
            # plot the roc curve
            plot_roc_curve(fpr, tpr, loc_acc, cl, lw=2)
            plt.savefig(f'{self.dir_name}/images/{iteration}_{cl}_roc.png')
            plt.close()
            
            # plot the confusion matrix
            conf = confusion_matrix(labels[cl], pred_label)
            plot_confusion_matrix(conf, [f'{cl}', 'healthy'])
            plt.savefig(f'{self.dir_name}/images/{iteration}_{cl}_cm.png')
            plt.close()
            

        acc["all"] = mean([acc[cl] for cl in self.classes])

        if "acc_all" in self.tr_dict.keys():
            for cl in acc.keys():
                self.tr_dict["acc_%s" % (cl)].append(acc[cl])
        else:
            for cl in acc.keys():
                self.tr_dict["acc_%s" % (cl)] = [acc[cl]]

        # plot result
        plt.figure(figsize=(12, 12))
        for k in self.tr_dict.keys():
            # Plot the current progress in "_Loss.png"
            plt.plot(self.tr_dict[k], label=k)

        # format plot
        plt.xlabel("Iteration")
        plt.ylabel("Loss")
        plt.grid(True, alpha=0.25)
        plt.legend()
        plt.savefig(self.dir_name + "/_Losses.pdf")
        plt.close()

        # write to "_Log.txt" file
        with open(self.dir_name + LOG, "a") as file:
            file.write("\n\nIteration: %d" % (iteration))
            for k in self.tr_dict.keys():
                file.write("\n%s: %.4f" % (k, self.tr_dict[k][-1]))

        # Save the losses as npy
        np.save(self.dir_name + "/_Losses.npy", self.tr_dict)

        # save the best performing networks
        for cl in acc.keys():
            self.update_network(cl, model, acc[cl])

In [ ]:
# export

from sklearn.metrics import accuracy_score, auc, confusion_matrix
from fastai.metrics import error_rate, accuracy, roc_curve, AUROC

def plot_roc_curve(fpr, tpr, auc, name, lw=2, show=False):
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange',
             lw=lw, label='ROC curve (area = %0.2f)' % auc)
    plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.grid(0.25)
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title(f'Receiver operating characteristic for {name}')
    plt.legend(loc="lower right")
    plt.show() if show else None

In [ ]:
# export

def plot_confusion_matrix(
    cm, target_names, title="Confusion matrix", cmap=None, normalize=False, show=False
):
    """
    Citiation
    ---------
    http://scikit-learn.org/stable/auto_examples/model_selection/plot_confusion_matrix.html

    """
    import matplotlib.pyplot as plt
    import numpy as np
    import itertools

    accuracy = np.trace(cm) / float(np.sum(cm))
    misclass = 1 - accuracy

    if cmap is None:
        cmap = plt.get_cmap("Blues")

    plt.figure(figsize=(8, 6))
    plt.imshow(cm, interpolation="nearest", cmap=cmap)
    plt.title(title)
    plt.colorbar()

    if target_names is not None:
        tick_marks = np.arange(len(target_names))
        plt.xticks(tick_marks, target_names, rotation=45)
        plt.yticks(tick_marks, target_names)

    if normalize:
        cm = cm.astype("float") / cm.sum(axis=1)[:, np.newaxis]

    thresh = cm.max() / 1.5 if normalize else cm.max() / 2
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        if normalize:
            plt.text(
                j,
                i,
                "{:0.4f}".format(cm[i, j]),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black",
            )
        else:
            plt.text(
                j,
                i,
                "{:,}".format(cm[i, j]),
                horizontalalignment="center",
                color="white" if cm[i, j] > thresh else "black",
            )

    plt.tight_layout()
    plt.ylabel("True label")
    plt.xlabel(
        "Predicted label\naccuracy={:0.4f}; misclass={:0.4f}".format(
            accuracy, misclass)
    )
    plt.show() if show else None

In [ ]:
# hide
from nbdev.export import notebook2script

notebook2script()

Converted 00_dataloader.ipynb.
Converted 01_architecture.ipynb.
Converted 02_utils.ipynb.
Converted 03_parameters.ipynb.
Converted 04_train_loop.ipynb.
Converted 05_abstract_model.ipynb.
Converted 10_diagnosis.ipynb.
Converted 20_dcgan.ipynb.
Converted 21_introvae.ipynb.
Converted 22_vqvae.ipynb.
Converted 23_bigan.ipynb.
Converted 24_mocoae.ipynb.
Converted 33_rnn_vae.ipynb.
Converted 99_index.ipynb.
